# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [5]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

from geoviews import opts

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [6]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("/Users/miracleonyeoziri/Documents/Bootcamp Materials/Module 6/Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(10)

# Renmaming first column to City_ID
vacation_df = vacation_df.rename(columns={"Unnamed: 0": "City_ID"})

# finding city_id 
# display(vacation_df.loc[15:100])

## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [7]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [8]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[[18],:]
vacation_end = vacation_df.loc[[18],:]
vacation_stop1 = vacation_df.loc[[43],:]
vacation_stop2 = vacation_df.loc[[0],:]
vacation_stop3 = vacation_df.loc[[59],:]

display(vacation_start)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,183,Vila Velha,BR,75.15,light rain,-20.3297,-40.2925,Hotel Prainha


## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [9]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], ignore_index=True, )

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,183,Vila Velha,BR,75.15,light rain,-20.3297,-40.2925,Hotel Prainha
1,435,Itanhem,BR,74.95,overcast clouds,-17.1664,-40.3300,Hotel Resende
2,0,Porto Seguro,BR,73.92,mist,-16.4497,-39.0647,Terra Mater Hotel
3,568,Unai,BR,73.56,overcast clouds,-16.3575,-46.9061,Hotel Fazenda Curva do Rio
4,183,Vila Velha,BR,75.15,light rain,-20.3297,-40.2925,Hotel Prainha


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [10]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]].copy()

# Display sample data
waypoints_df

,Lat,Lng
0,-20.3297,-40.2925
1,-17.1664,-40.3300
2,-16.4497,-39.0647
3,-16.3575,-46.9061
4,-20.3297,-40.2925


## Use GeoViews to create map that shows the four cities in the itinerary

In [207]:
# Configure the map plot by using the itinerary_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Max Temp",
    scale = 2,
    color = "City",
    alpha = 0.5 #adjusting the opacty of the colored dots
)

In [208]:
# Display the route_map
waypoints_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [13]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [14]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the waypoints_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'fdefb5b1ac4d453896f82d29aa11746f',
 'waypoints': '-20.3297,-40.2925|-17.1664,-40.33|-16.4497,-39.0647|-16.3575,-46.9061|-20.3297,-40.2925'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [15]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [16]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [ ]:
# Create an empty list to store the longitude of each step
longitude = ""

# Create an empty list to store the latitude of each step
latitude = ""

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for coordinates_set in legs: #loops through each set of coordinates in the legs list (there are 4 sets)
    for coordinate in coordinates_set: #nested loop iterates through each individual coordinate in the set
        latitude, longitude = coordinate #unpacks lat and long from each coordinate
        print(f'Latitude: {latitude}, Longitude: {longitude}') #placeholder

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [18]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
for coordinates_set in legs:
    #Looping through each individual coordinate in the set
    for coordinate in coordinates_set:
        #Unpacking the latitude and longitude from the coordinate
        latitude, longitude = coordinate
        #Appending a new row to route_df with latitude and longitude
        route_df = route_df.append({'Latitude': latitude, 'Longitude': longitude}, ignore_index=True)

# Display sample data
route_df.head(5)

,Latitude,Longitude
0,-40.292785,-20.329689
1,-40.292787,-20.329731
2,-40.292794,-20.329909
3,-40.292792,-20.329927
4,-40.292787,-20.329942


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [212]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).opts(width=600, height=400, color="red", line_width=2)

In [214]:
# Display a composed plot by using the route_map and route_path objects
route_map = gv.tile_sources.OSM * waypoints_map * route_path #adjust the order to change the order of overlaying
route_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .WMTS.II  :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)
   .Path.I   :Path   [Latitude,Longitude]